# AlphaVantage API

### Initialize

In [2]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta
import time

In [12]:
load_dotenv()

apikey = os.getenv("apikey")

In [4]:
base_url = "https://www.alphavantage.co/query?"

In [5]:
symbols = [
    'IBM',    # Technology / Services
    'AAPL',   # Apple - Technology
    'MSFT',   # Microsoft - Technology
    'AMZN',   # Amazon - Consumer Goods / Technology
    'GOOGL',  # Alphabet (Google) - Technology
    'META',     # Meta (Facebook) - Technology
    'JPM',    # JPMorgan Chase - Financial
    'V',      # Visa - Financial
    'PG',     # Procter & Gamble - Consumer Goods
    'UNH',    # UnitedHealth - Healthcare
    'JNJ',    # Johnson & Johnson - Healthcare
    'XOM',    # ExxonMobil - Energy
    'CVX',    # Chevron - Energy
    'TSLA',   # Tesla - Automotive / Energy
    'WMT',    # Walmart - Retail
    'BAC',    # Bank of America - Financial
    'INTC',   # Intel - Technology
    'CSCO',   # Cisco Systems - Technology
    'KO',     # Coca-Cola - Consumer Goods
    'PEP',    # PepsiCo - Consumer Goods
    'MCD',    # McDonald's - Consumer Services
    'MMM',    # 3M - Conglomerate
    'BA',     # Boeing - Aerospace
    'CAT',    # Caterpillar - Heavy Equipment
    'LMT',    # Lockheed Martin - Aerospace and Defense
    'GS',     # Goldman Sachs - Financial
    'DIS',    # Disney - Media and Entertainment
    'PFE',    # Pfizer - Healthcare
    'MRK',    # Merck - Healthcare
    'GE',     # General Electric - Conglomerate
    'NKE',    # Nike - Apparel
    'SBUX',   # Starbucks - Specialty Eateries
    'NFLX',   # Netflix - Media
    'ORCL',   # Oracle - Technology
    'AMD',    # Advanced Micro Devices - Technology
    'NVDA',   # NVIDIA - Technology
    'F',      # Ford - Automotive
    'GM',     # General Motors - Automotive
    'UPS',    # United Parcel Service - Logistics
    'FDX',    # FedEx - Logistics
    'LYFT',   # Lyft - Ride-Sharing
    'UBER'    # Uber - Ride-Sharing
]

### Balance Sheet

In [6]:
dfs = []
for symbol in symbols:
    balance_sheet_url = f"{base_url}function=BALANCE_SHEET&symbol={symbol}&apikey={apikey}"
    balance_sheet_response = requests.get(balance_sheet_url)

    # Check if the request was successful
    if balance_sheet_response.status_code == 200:
        # Parse JSON response
        balance_sheet_data = balance_sheet_response.json()
        
        # Initialize lists to collect data
        fiscal_date_endings = []
        total_assets = []
        total_liabilities = []
        total_shareholder_equity = []
        
        # Extract quarterly reports data
        for report in balance_sheet_data["quarterlyReports"]:
            fiscal_date_endings.append(report['fiscalDateEnding'])
            total_assets.append(report['totalAssets'])
            total_liabilities.append(report.get('totalLiabilities', ""))
            total_shareholder_equity.append(report.get('totalShareholderEquity', ""))
        
        # Create DataFrame for the current symbol
        quarterly_reports_df = pd.DataFrame({
            'Symbol': [symbol] * len(fiscal_date_endings),
            'Fiscal Date Ending': fiscal_date_endings,
            'Total Assets': total_assets,
            'Total Liabilities': total_liabilities,
            'Total Shareholder Equity': total_shareholder_equity
        })
        
        # Append the DataFrame to the list
        dfs.append(quarterly_reports_df)

# Concatenate all DataFrames in the list
quarterly_reports_df = pd.concat(dfs, ignore_index=True)

quarterly_reports_df

,Symbol,Fiscal Date Ending,Total Assets,Total Liabilities,Total Shareholder Equity
0,IBM,2024-03-31,137169000000,113836000000,23261000000
1,IBM,2023-12-31,135241000000,112628000000,22533000000
2,IBM,2023-09-30,129321000000,106165000000,23081000000
3,IBM,2023-06-30,132213000000,109942000000,22201000000
4,IBM,2023-03-31,133637000000,111964000000,21604000000
...,...,...,...,...,...
2330,UBER,2020-03-31,30090000000,17772000000,11342000000
2331,UBER,2019-12-31,31761000000,16578000000,14190000000
2332,UBER,2019-09-30,32292000000,16241000000,15062000000
2333,UBER,2019-06-30,30980000000,15072000000,15922000000


### Income Statement

In [7]:
dfs = []

for symbol in symbols:
    income_statement_url = f"{base_url}function=INCOME_STATEMENT&symbol={symbol}&apikey={apikey}"
    try:
        income_statement_response = requests.get(income_statement_url)
        income_statement_response.raise_for_status()  # Raises an HTTPError for bad responses
        income_statement_data = income_statement_response.json()
    except requests.RequestException as e:
        print(f"Request failed for {symbol}: {e}")
        continue
    
    fiscal_date_endings = []
    total_revenue = []
    total_expenses = []
    ebit = []
    
    for income_report in income_statement_data["quarterlyReports"]:
        fiscal_date_endings.append(income_report['fiscalDateEnding'])
        
        # Safely convert revenue to float
        try:
            total_revenue.append(float(income_report['totalRevenue']))
        except ValueError:
            total_revenue.append(0)
        
        # Safely calculate total expenses
        expenses_components = ['costOfRevenue', 'operatingExpenses', 'interestAndDebtExpense',
                               'depreciationAndAmortization', 'incomeTaxExpense']
        expense_total = 0
        for component in expenses_components:
            try:
                expense_total += float(income_report.get(component, 0) or 0)
            except ValueError:
                continue  # Skip component if conversion fails
        
        total_expenses.append(expense_total)
        ebit.append(income_report.get('ebit', ""))
    
    # Create DataFrame for the current symbol
    income_statements_df = pd.DataFrame({
        'Symbol': [symbol] * len(fiscal_date_endings),
        'Fiscal Date Ending': fiscal_date_endings,
        'Total Revenue': total_revenue,
        'Total Expenses': total_expenses,
        'EBIT': ebit
    })
    
    # Append the DataFrame to the list
    dfs.append(income_statements_df)

# Concatenate all DataFrames in the list
income_statements_df = pd.concat(dfs, ignore_index=True)

income_statements_df

,Symbol,Fiscal Date Ending,Total Revenue,Total Expenses,EBIT
0,IBM,2024-03-31,1.446200e+10,1.490400e+10,1535000000
1,IBM,2023-12-31,1.738100e+10,1.499500e+10,4167000000
2,IBM,2023-09-30,1.475200e+10,1.390100e+10,2275000000
3,IBM,2023-06-30,1.547500e+10,1.489000e+10,2425000000
4,IBM,2023-03-31,1.425200e+10,1.503400e+10,1418000000
...,...,...,...,...,...
2491,UBER,2019-03-31,3.099000e+09,5.417000e+09,-776000000
2492,UBER,2018-12-31,2.137000e+09,3.814000e+09,-1014000000
2493,UBER,2018-09-30,2.944000e+09,3.839000e+09,-824000000
2494,UBER,2018-06-30,2.768000e+09,3.633000e+09,-690000000


### Cash Flow

In [8]:
dfs = []  

for symbol in symbols:
    cash_flow_url = f"{base_url}function=CASH_FLOW&symbol={symbol}&apikey={apikey}"
    try:
        cash_flow_response = requests.get(cash_flow_url)
        cash_flow_response.raise_for_status() 
        cash_flow_data = cash_flow_response.json()
    except requests.RequestException as e:
        print(f"Request failed for {symbol}: {e}")
        continue
    
    fiscal_date_endings = []
    investing_flows = []
    operating_flows = []
    financing_flows = []
    
    if "quarterlyReports" in cash_flow_data:
        for cash_flow_report in cash_flow_data["quarterlyReports"]:
            fiscal_date_endings.append(cash_flow_report['fiscalDateEnding'])

            try:
                financing_cash_flow = float(cash_flow_report.get("cashflowFromFinancing", 0) or 0)
                dividend_payout = float(cash_flow_report.get("dividendPayout", 0) or 0)
                financing_flows.append(financing_cash_flow + dividend_payout)
            except ValueError:
                financing_flows.append(0)
            
            try:
                capital_expenditures = float(cash_flow_report.get("capitalExpenditures", 0) or 0)
                investment_cash_flow = float(cash_flow_report.get("cashflowFromInvestment", 0) or 0)
                investing_flows.append(capital_expenditures + investment_cash_flow)
            except ValueError:
                investing_flows.append(0)
            
            try:
                operating_flows.append(float(cash_flow_report.get('operatingCashflow', 0) or 0))
            except ValueError:
                operating_flows.append(0)
        
        cash_flows_df = pd.DataFrame({
            'Symbol': [symbol] * len(fiscal_date_endings),
            'Fiscal Date Ending': fiscal_date_endings,
            'Operating Cash Flow': operating_flows,
            'Investing Cash Flow': investing_flows,
            'Financing Cash Flow': financing_flows
        })

        dfs.append(cash_flows_df)

all_cash_flows_df = pd.concat(dfs, ignore_index=True)
all_cash_flows_df 

,Symbol,Fiscal Date Ending,Operating Cash Flow,Investing Cash Flow,Financing Cash Flow
0,IBM,2024-03-31,4.168000e+09,-3.971000e+09,3.399000e+09
1,IBM,2023-12-31,4.463000e+09,3.136000e+09,-9.700000e+07
2,IBM,2023-09-30,3.056000e+09,-1.672000e+09,-1.617000e+09
3,IBM,2023-06-30,2.638000e+09,3.710000e+08,-1.220000e+09
4,IBM,2023-03-31,3.774000e+09,-7.660000e+09,7.205000e+09
...,...,...,...,...,...
2399,UBER,2019-03-31,-7.220000e+08,3.330000e+08,0.000000e+00
2400,UBER,2018-12-31,-6.810000e+08,4.000000e+07,0.000000e+00
2401,UBER,2018-09-30,-4.100000e+08,8.000000e+07,0.000000e+00
2402,UBER,2018-06-30,-1.530000e+08,2.800000e+07,0.000000e+00


### Market Indicators

RSI:

In [17]:
load_dotenv()

apikey = os.getenv("apikey")

def fetch_rsi_daily(base_url, apikey, symbols, start_year, start_month):
    all_rsi_data = pd.DataFrame()
    
    for symbol in symbols:
        start_date = datetime(start_year, start_month, 1)
        current_date = datetime.now().replace(day=1)
        
        while start_date < current_date:
            month_str = start_date.strftime('%Y-%m')
            rsi_url = f'{base_url}function=RSI&symbol={symbol}&interval=daily&time_period=10&series_type=open&apikey={apikey}&month={month_str}'
            try:
                response = requests.get(rsi_url)
                response.raise_for_status()
                data = response.json()

                if 'Technical Analysis: RSI' in data:
                    rsi_data = data['Technical Analysis: RSI']
                    df = pd.DataFrame.from_dict(rsi_data, orient='index')
                    df.index = pd.to_datetime(df.index)
                    df.columns = ['Daily RSI']
                    df['Daily RSI'] = df['Daily RSI'].astype(float)
                    df['Symbol'] = symbol  
                    
                    all_rsi_data = pd.concat([all_rsi_data, df])
                else:
                    print(f"No RSI data available for {symbol} in {month_str}")
            except requests.RequestException as e:
                print(f"Failed to retrieve data for {symbol} in {month_str}: {e}")
            
            start_date = (start_date + timedelta(days=32)).replace(day=1)
            time.sleep(1) 
    
    return all_rsi_data

daily_rsi_data_df = fetch_rsi_daily(base_url, apikey, symbols, 2000, 1)

daily_rsi_data_df

VWAP:

In [14]:
load_dotenv()

apikey = os.getenv("apikey")

def fetch_vwap_daily(base_url, apikey, symbols, start_year, start_month):
    all_vwap_data = pd.DataFrame()

    for symbol in symbols:
        start_date = datetime(start_year, start_month, 1)
        current_date = datetime.now().replace(day=1)
        
        while start_date < current_date:
            month_str = start_date.strftime('%Y-%m')
            vwap_url = f"{base_url}function=VWAP&symbol={symbol}&interval=60min&apikey={apikey}&month={month_str}"
            try:
                response = requests.get(vwap_url)
                response.raise_for_status()
                data = response.json()
            except requests.RequestException as e:
                print(f"Failed to retrieve VWAP data for {symbol} in {month_str}: {e}")
                continue

            if 'Technical Analysis: VWAP' in data:
                vwap_data = data['Technical Analysis: VWAP']
                df = pd.DataFrame.from_dict(vwap_data, orient='index')
                df.index = pd.to_datetime(df.index)
                df.columns = ['Daily VWAP']
                df['Daily VWAP'] = df['Daily VWAP'].astype(float)

                df['Symbol'] = symbol
                all_vwap_data = pd.concat([all_vwap_data, df])

            else:
                print(f"No VWAP data available for {symbol} in {month_str}")

            start_date += timedelta(days=32)
            start_date = start_date.replace(day=1)

            time.sleep(1)

    return all_vwap_data

daily_vwap_data = fetch_vwap_daily(base_url, apikey, symbols, 2000, 1)
daily_vwap_data

KeyboardInterrupt: 

MACD:

In [ ]:
load_dotenv()

apikey = os.getenv("apikey")

def fetch_macd_daily(base_url, apikey, symbols, start_year, start_month):
    all_macd_data = pd.DataFrame() 

    for symbol in symbols: 
        start_date = datetime(start_year, start_month, 1)
        current_date = datetime.now().replace(day=1)

        while start_date < current_date:
            month_str = start_date.strftime('%Y-%m')
            macd_url = f'{base_url}function=MACD&symbol={symbol}&interval=daily&time_period=10&series_type=open&apikey={apikey}&month={month_str}'
            try:
                response = requests.get(macd_url)
                response.raise_for_status() 
                data = response.json()
            except requests.RequestException as e:
                print(f"Failed to retrieve MACD data for {symbol} in {month_str}: {e}")
                continue 

            if 'Technical Analysis: MACD' in data:
                macd_data = data['Technical Analysis: MACD']
                df = pd.DataFrame({date: values['MACD'] for date, values in macd_data.items()}, index=[0]).T
                df.index = pd.to_datetime(df.index)
                df.columns = ['Daily MACD']
                df['Daily MACD'] = df['Daily MACD'].astype(float)
                df['Symbol'] = symbol 

                all_macd_data = pd.concat([all_macd_data, df])
            else:
                print(f"No MACD data available for {symbol} in {month_str}")

            start_date += timedelta(days=32)
            start_date = start_date.replace(day=1)

            time.sleep(1) 
            
    return all_macd_data

daily_macd_data_df = fetch_macd_daily(base_url, apikey, symbols, 2000, 1)

daily_macd_data_df

,Daily MACD
2024-05-10,-4.9744
2024-05-09,-5.2641
2024-05-08,-5.5950
2024-05-07,-5.8177
2024-05-06,-6.1173
...,...
1999-12-23,1.5658
1999-12-22,1.7042
1999-12-21,1.8167
1999-12-20,2.0058


Daily Technical Indicators:

In [ ]:
start_date = datetime(2000, 1, 1)
end_date = datetime.now().replace(day=1)

daily_macd_data_df = daily_macd_data_df[(daily_macd_data_df.index >= start_date) & (daily_macd_data_df.index < end_date)]
daily_vwap_data = daily_vwap_data[(daily_vwap_data.index >= start_date) & (daily_vwap_data.index < end_date)]
daily_rsi_data_df = daily_rsi_data_df[(daily_rsi_data_df.index >= start_date) & (daily_rsi_data_df.index < end_date)]

daily_macd_data_df.index = daily_macd_data_df.index.normalize()
daily_vwap_data.index = daily_vwap_data.index.normalize()
daily_rsi_data_df.index = daily_rsi_data_df.index.normalize()

combined_df = pd.merge(daily_macd_data_df, daily_vwap_data, how='inner', left_index=True, right_index=True)
combined_df = pd.merge(combined_df, daily_rsi_data_df, how='inner', left_index=True, right_index=True)

daily_technical_indicators_combined_df = combined_df[~combined_df.index.duplicated(keep='first')]

daily_technical_indicators_combined_df

,Daily MACD,Daily VWAP,Daily RSI
2024-01-02,2.4353,162.4933,61.4287
2024-01-03,2.1528,159.5297,48.9172
2024-01-04,1.8459,158.8627,44.6138
2024-01-05,1.5602,157.5377,42.9455
2024-01-08,1.2230,157.4423,36.9099
...,...,...,...
2024-04-25,-2.9381,166.6323,14.6350
2024-04-26,-3.8716,167.6987,14.1911
2024-04-29,-4.5667,165.7713,14.1232
2024-04-30,-5.1311,165.5237,13.4707


Quarterly Technical Indicators:

### Ratios

Debt to Equity Ratio:

In [ ]:
balance_sheet_url = f"{base_url}function=BALANCE_SHEET&symbol={symbol}&apikey={apikey}"
balance_sheet_response = requests.get(balance_sheet_url)

if balance_sheet_response.status_code == 200:
    balance_sheet_data = balance_sheet_response.json()
    
    fiscal_date_endings = []
    total_assets = []
    total_liabilities = []
    total_shareholder_equity = []
    
    for report in balance_sheet_data["quarterlyReports"]:
        fiscal_date_endings.append(report['fiscalDateEnding'])
        total_liabilities.append(float(report.get('totalLiabilities', "0")))
        total_shareholder_equity.append(float(report.get('totalShareholderEquity', "0")))
    
    debt_to_equity_ratios = [liabilities / equity if equity != 0 else 0 for liabilities, equity in zip(total_liabilities, total_shareholder_equity)]
    
    debt_to_equity_ratio_df = pd.DataFrame({
        'Symbol': [symbol] * len(fiscal_date_endings),
        'Fiscal Date Ending': fiscal_date_endings,
        'Debt to Equity Ratio': debt_to_equity_ratios
    })

debt_to_equity_ratio_df

,Symbol,Fiscal Date Ending,Debt to Equity Ratio
0,IBM,2024-03-31,4.893857
1,IBM,2023-12-31,4.998358
2,IBM,2023-09-30,4.599671
3,IBM,2023-06-30,4.952119
4,IBM,2023-03-31,5.182559
5,IBM,2022-12-31,4.795024
6,IBM,2022-09-30,5.265142
7,IBM,2022-06-30,5.565768
8,IBM,2022-03-31,5.992756
9,IBM,2021-12-31,5.978784


Inventory Turnover

In [ ]:

def fetch_financial_data(base_url, apikey, symbol):
    balance_sheet_url = f"{base_url}function=BALANCE_SHEET&symbol={symbol}&apikey={apikey}"
    income_statement_url = f"{base_url}function=INCOME_STATEMENT&symbol={symbol}&apikey={apikey}"

    balance_response = requests.get(balance_sheet_url)
    balance_data = balance_response.json() if balance_response.status_code == 200 else {}

    income_response = requests.get(income_statement_url)
    income_data = income_response.json() if income_response.status_code == 200 else {}

    fiscal_date_endings = []
    inventory_turnovers = []

    if 'quarterlyReports' in balance_data and 'quarterlyReports' in income_data:
        balance_reports = balance_data['quarterlyReports']
        income_reports = {report['fiscalDateEnding']: float(report['costofGoodsAndServicesSold']) for report in income_data['quarterlyReports']}

        for report in balance_reports:
            date = report['fiscalDateEnding']
            if date in income_reports:
                inventory = float(report.get('inventory', '0'))
                cogs = income_reports[date]

                current_index = balance_reports.index(report)
                previous_inventory = float(balance_reports[current_index - 1].get('inventory', '0')) if current_index > 0 else inventory
                average_inventory = (inventory + previous_inventory) / 2

                inventory_turnover = cogs / average_inventory if average_inventory != 0 else 0
                fiscal_date_endings.append(date)
                inventory_turnovers.append(inventory_turnover)

    inventory_turnover_df = pd.DataFrame({
        'Symbol': [symbol] * len(fiscal_date_endings),
        'Fiscal Date Ending': fiscal_date_endings,
        'Inventory Turnover': inventory_turnovers
    })

    return inventory_turnover_df

inventory_turnover_df = fetch_financial_data(base_url, apikey, 'IBM')
inventory_turnover_df


,Symbol,Fiscal Date Ending,Inventory Turnover
0,IBM,2024-03-31,0.023102
1,IBM,2023-12-31,0.094395
2,IBM,2023-09-30,0.032813
3,IBM,2023-06-30,0.042069
4,IBM,2023-03-31,0.019330
5,IBM,2022-12-31,0.120444
6,IBM,2022-09-30,0.034071
7,IBM,2022-06-30,0.069005
8,IBM,2022-03-31,0.010983
9,IBM,2021-12-31,0.051971


ROI

In [ ]:

def fetch_roa_data(base_url, apikey, symbol):
    balance_sheet_url = f"{base_url}function=BALANCE_SHEET&symbol={symbol}&apikey={apikey}"
    income_statement_url = f"{base_url}function=INCOME_STATEMENT&symbol={symbol}&apikey={apikey}"

    balance_response = requests.get(balance_sheet_url)
    balance_data = balance_response.json() if balance_response.status_code == 200 else {}

    income_response = requests.get(income_statement_url)
    income_data = income_response.json() if income_response.status_code == 200 else {}

    fiscal_date_endings = []
    roa_values = []

    if 'quarterlyReports' in balance_data and 'quarterlyReports' in income_data:
        balance_reports = {report['fiscalDateEnding']: float(report['totalAssets']) for report in balance_data['quarterlyReports']}
        income_reports = {report['fiscalDateEnding']: float(report['netIncome']) for report in income_data['quarterlyReports']}

        for date, assets in balance_reports.items():
            if date in income_reports:
                net_income = income_reports[date]
                roa = (net_income / assets) * 100 if assets != 0 else 0
                fiscal_date_endings.append(date)
                roa_values.append(roa)

    roa_df = pd.DataFrame({
        'Symbol': [symbol] * len(fiscal_date_endings),
        'Fiscal Date Ending': fiscal_date_endings,
        'ROA (%)': roa_values
    })

    return roa_df

roa_df = fetch_roa_data(base_url, apikey, 'IBM')
roa_df

,Symbol,Fiscal Date Ending,ROA (%)
0,IBM,2024-03-31,1.170089
1,IBM,2023-12-31,2.431215
2,IBM,2023-09-30,1.317651
3,IBM,2023-06-30,1.197310
4,IBM,2023-03-31,0.693670
5,IBM,2022-12-31,2.129783
6,IBM,2022-09-30,-2.539531
7,IBM,2022-06-30,1.091739
8,IBM,2022-03-31,0.549991
9,IBM,2021-12-31,1.767411


### Economic Indicators

Inflation

In [ ]:
inflation_url = f"{base_url}function=INFLATION&symbol={symbol}&apikey={apikey}"
inflation_response = requests.get(inflation_url)

if inflation_response.status_code == 200:
    inflation_data = inflation_response.json()
    
    report_date = []
    inflation_rate = []
    
    for value in inflation_data["data"]:
        report_date.append(value['date'])
        inflation_rate.append(value['value'])
    
    inflation_rates_df = pd.DataFrame({
        'Date': report_date,
        'Inflation Rate': inflation_rate
    })

inflation_rates_df

,Date,Inflation Rate
0,2022-01-01,8.00279982052121
1,2021-01-01,4.69785886363742
2,2020-01-01,1.23358439630629
3,2019-01-01,1.81221007526021
4,2018-01-01,2.44258329692817
...,...,...
58,1964-01-01,1.27891156462591
59,1963-01-01,1.23966942148753
60,1962-01-01,1.19877334820186
61,1961-01-01,1.07072414764724


Unemployment

In [ ]:
unemployment_url = f"{base_url}function=UNEMPLOYMENT&symbol={symbol}&apikey={apikey}"
unemployment_response = requests.get(unemployment_url)

if unemployment_response.status_code == 200:
    unemployment_data = unemployment_response.json()
    
    report_date = []
    unemployment_rate = []
    
    for value in unemployment_data["data"]:
        report_date.append(value['date'])
        unemployment_rate.append(value['value'])
    
    unemployment_rates_df = pd.DataFrame({
        'Date': report_date,
        'Unemployment Rate': unemployment_rate
    })

unemployment_rates_df

,Date,Unemployment Rate
0,2024-04-01,3.9
1,2024-03-01,3.8
2,2024-02-01,3.9
3,2024-01-01,3.7
4,2023-12-01,3.7
...,...,...
911,1948-05-01,3.5
912,1948-04-01,3.9
913,1948-03-01,4.0
914,1948-02-01,3.8


GPD

In [ ]:
gdp_url = f"{base_url}function=GDP&symbol={symbol}&apikey={apikey}"
gdp_response = requests.get(gdp_url)

if gdp_response.status_code == 200:
    gdp_data = unemployment_response.json()
    
    report_date = []
    gdp_rate = []
    
    for value in gdp_data["data"]:
        report_date.append(value['date'])
        gdp_rate.append(value['value'])
    
    gdp_rates_df = pd.DataFrame({
        'Date': report_date,
        'GDP Rate': gdp_rate
    })

gdp_rates_df

,Date,GDP Rate
0,2024-04-01,3.9
1,2024-03-01,3.8
2,2024-02-01,3.9
3,2024-01-01,3.7
4,2023-12-01,3.7
...,...,...
911,1948-05-01,3.5
912,1948-04-01,3.9
913,1948-03-01,4.0
914,1948-02-01,3.8


## Company Overview

In [ ]:
dfs = []

for symbol in symbols:
    company_overview_url = f"{base_url}function=OVERVIEW&symbol={symbol}&apikey={apikey}"
    company_overview_response = requests.get(company_overview_url)

    if company_overview_response.status_code == 200:
        company_overview_data = company_overview_response.json()

        company_overview_selected_data = {
            "Symbol": company_overview_data.get("Symbol", ""),
            "Name": company_overview_data.get("Name", ""),
            "Industry": company_overview_data.get("Industry", ""),
            "Address": company_overview_data.get("Address", "")
        }

        temp_df = pd.DataFrame([company_overview_selected_data])
        address_parts = temp_df['Address'].str.split(', ', expand=True)
        temp_df['Street Address'] = address_parts.get(0, "")
        temp_df['City'] = address_parts.get(1, "")
        temp_df['State'] = address_parts.get(2, "")
        temp_df['Country'] = address_parts.get(3, "")
        temp_df = temp_df.drop(columns=['Address'])

        dfs.append(temp_df)

company_overview_df = pd.concat(dfs, ignore_index=True)

company_overview_df = company_overview_df.dropna(how='all')

company_overview_df

,Symbol,Name,Industry,Street Address,City,State,Country
0,IBM,International Business Machines,COMPUTER & OFFICE EQUIPMENT,1 NEW ORCHARD ROAD,ARMONK,NY,US
1,AAPL,Apple Inc,ELECTRONIC COMPUTERS,ONE INFINITE LOOP,CUPERTINO,CA,US
2,MSFT,Microsoft Corporation,SERVICES-PREPACKAGED SOFTWARE,ONE MICROSOFT WAY,REDMOND,WA,US
3,AMZN,Amazon.com Inc,RETAIL-CATALOG & MAIL-ORDER HOUSES,410 TERRY AVENUE NORTH,SEATTLE,WA,US
4,GOOGL,Alphabet Inc Class A,"SERVICES-COMPUTER PROGRAMMING, DATA PROCESSING...",1600 AMPHITHEATRE PARKWAY,MOUNTAIN VIEW,CA,US
5,META,Meta Platforms Inc.,"SERVICES-COMPUTER PROGRAMMING, DATA PROCESSING...",1601 WILLOW ROAD,MENLO PARK,CA,US
6,JPM,JPMorgan Chase & Co,NATIONAL COMMERCIAL BANKS,383 MADISON AVENUE,NEW YORK,NY,US
7,V,Visa Inc. Class A,"SERVICES-BUSINESS SERVICES, NEC",P.O. BOX 8999,SAN FRANCISCO,CA,US
8,PG,Procter & Gamble Company,"SOAP, DETERGENTS, CLEANG PREPARATIONS, PERFUME...",ONE PROCTER & GAMBLE PLAZA,CINCINNATI,OH,US
9,UNH,UnitedHealth Group Incorporated,HOSPITAL & MEDICAL SERVICE PLANS,UNITEDHEALTH GROUP CENTER,9900 BREN ROAD EAST,MINNEAPOLIS,MN


## Financial Statements

In [ ]:
quarterly_reports_df['Fiscal Date Ending'] = pd.to_datetime(quarterly_reports_df['Fiscal Date Ending'])
income_statements_df['Fiscal Date Ending'] = pd.to_datetime(income_statements_df['Fiscal Date Ending'])
cash_flows_df['Fiscal Date Ending'] = pd.to_datetime(cash_flows_df['Fiscal Date Ending'])

combined_df = pd.merge(quarterly_reports_df, income_statements_df, on=['Symbol', 'Fiscal Date Ending'], how='outer')
combined_df = pd.merge(combined_df, cash_flows_df, on=['Symbol', 'Fiscal Date Ending'], how='outer')

financial_statements_df = combined_df[combined_df['Fiscal Date Ending'] >= '2009-06-30']

financial_statements_df.sort_values('Fiscal Date Ending', inplace=True)

financial_statements_df

/var/folders/kf/x49nrq891g3dcplsvlw9bh4r0000gn/T/ipykernel_10058/3766564765.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  financial_statements_df.sort_values('Fiscal Date Ending', inplace=True)


,Symbol,Fiscal Date Ending,Total Assets,Total Liabilities,Total Shareholder Equity,Total Revenue,Total Expenses,EBIT,Operating Cash Flow,Investing Cash Flow,Financing Cash Flow
59,IBM,2009-06-30,103655000000,88182000000,15380000000,23250000000,2.182100e+10,4363000000,4.741000e+09,-9.860000e+08,-3.011000e+09
58,IBM,2009-09-30,103675000000,85182000000,18396000000,23566000000,2.102900e+10,4457000000,5.198000e+09,-2.180000e+09,-4.442000e+09
57,IBM,2009-12-31,109022000000,86267000000,22637000000,27230000000,2.343100e+10,6462000000,6.448000e+09,1.106000e+09,-4.790000e+08
56,IBM,2010-03-31,105208000000,83059000000,22033000000,22857000000,2.171600e+10,3597000000,4.437000e+09,-5.530000e+08,-2.650000e+09
55,IBM,2010-06-30,103420000000,82244000000,21059000000,23724000000,2.152000e+10,4665000000,3.766000e+09,-1.805000e+09,-2.913000e+09
54,IBM,2010-09-30,107174000000,84815000000,22238000000,24271000000,2.099800e+10,4769400000,4.551000e+09,-1.634000e+09,-2.637000e+09
53,IBM,2010-12-31,113452000000,90279000000,23046000000,29018000000,2.674600e+10,7052400000,6.795000e+09,2.390000e+08,-1.052000e+09
52,IBM,2011-03-31,112960000000,90185000000,22671000000,24607000000,2.347800e+10,3907400000,3.792000e+09,1.556000e+09,-1.519000e+09
51,IBM,2011-06-30,113474000000,90263000000,23127000000,26666000000,2.422500e+10,4979900000,4.279000e+09,3.670000e+08,-3.998000e+09
50,IBM,2011-09-30,110158000000,87781000000,22291000000,26157000000,2.351100e+10,5133100000,3.839000e+09,6.670000e+08,-2.776000e+09


## Key Financial Ratios

In [ ]:
combined_df = pd.merge(debt_to_equity_ratio_df, inventory_turnover_df, on=['Symbol', 'Fiscal Date Ending'], how='inner')

key_financial_ratios_df = pd.merge(combined_df, roa_df, on=['Symbol', 'Fiscal Date Ending'], how='inner')

key_financial_ratios_df

,Symbol,Fiscal Date Ending,Debt to Equity Ratio,Inventory Turnover,ROA (%)
0,IBM,2024-03-31,4.893857,0.023102,1.170089
1,IBM,2023-12-31,4.998358,0.094395,2.431215
2,IBM,2023-09-30,4.599671,0.032813,1.317651
3,IBM,2023-06-30,4.952119,0.042069,1.197310
4,IBM,2023-03-31,5.182559,0.019330,0.693670
5,IBM,2022-12-31,4.795024,0.120444,2.129783
6,IBM,2022-09-30,5.265142,0.034071,-2.539531
7,IBM,2022-06-30,5.565768,0.069005,1.091739
8,IBM,2022-03-31,5.992756,0.010983,0.549991
9,IBM,2021-12-31,5.978784,0.051971,1.767411


## Accounts

In [ ]:
balance_sheet_url = f"{base_url}function=BALANCE_SHEET&symbol={symbol}&apikey={apikey}"
balance_sheet_response = requests.get(balance_sheet_url)

if balance_sheet_response.status_code == 200:
    balance_sheet_data = balance_sheet_response.json()
    
    fiscal_date_endings = []
    current_net_receivables = []
    current_accounts_payable = []
    
    for report in balance_sheet_data["quarterlyReports"]:
        fiscal_date_endings.append(report['fiscalDateEnding'])
        current_net_receivables.append(report['currentNetReceivables'])
        current_accounts_payable.append(report['currentAccountsPayable'])
    
    quarterly_accounts_df = pd.DataFrame({
        'Symbol': [symbol] * len(fiscal_date_endings),
        'Fiscal Date Ending': fiscal_date_endings,
        'Current Net Receivables': current_net_receivables,
        'Current Accounts Payable': current_accounts_payable
    })

quarterly_accounts_df

## Market Indicators

In [ ]:
quarterly_market_indicators_data_df = daily_technical_indicators_combined_df.resample('Q').mean()

rename_dict = {col: col.replace('Daily', 'Quarterly') for col in quarterly_market_indicators_data_df.columns}

quarterly_market_indicators_data_df.rename(columns=rename_dict, inplace=True)

quarterly_market_indicators_data_df

,Quarterly MACD,Quarterly VWAP,Quarterly RSI
2024-03-31,3.694389,180.999116,63.159367
2024-06-30,-1.210739,180.285335,34.693057


## Economic Indicators

In [ ]:
gdp_rates_df['Date'] = pd.to_datetime(gdp_rates_df['Date'])
inflation_rates_df['Date'] = pd.to_datetime(inflation_rates_df['Date'])
unemployment_rates_df['Date'] = pd.to_datetime(unemployment_rates_df['Date'])

combined_df = pd.merge(gdp_rates_df, inflation_rates_df, on='Date', how='inner', suffixes=('', '_inflation'))

combined_df = pd.merge(combined_df, unemployment_rates_df, on='Date', how='inner', suffixes=('', '_unemployment'))

economic_indicators_df = combined_df[combined_df['Date'] >= '1960-01-01']

economic_indicators_df.rename(columns={
    'GDP Rate': 'GDP',
    'value': 'Inflation Rate',
    'value_unemployment': 'Unemployment Rate'
}, inplace=True)

economic_indicators_df.sort_values('Date', inplace=True)

economic_indicators_df